# De la représentation au machine learning

Un exemple parmi d'autres

- une représentation numérique
- un modèle (supevisé ou non supervisé) en fonction de la tâche

## Charger les données

In [ ]:
import pandas as pd
df = pd.read_csv("../data/dataframe.csv")
df.head()

## Calculer une représentation tf-idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt


vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['texte'])

## Faire une classification hiérarchique ascendante


In [ ]:
Z = linkage(tfidf_matrix.toarray(), method='ward')  #

Faire une représentation

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(5, 5))
dendrogram(Z, orientation='right')
plt.title("Classification Hiérarchique Ascendante (CHA)")
plt.tight_layout()
plt.show()

Quels sont les questions les plus fréquentes ? Sont-elles posées sur les mêmes périodes ?

In [ ]:
from sklearn.cluster import AgglomerativeClustering
cluster = AgglomerativeClustering(n_clusters=12,  linkage='ward')
labels = cluster.fit_predict(tfidf_matrix.toarray())
df['cluster'] = labels

Faire une représentation temporelle des clusters

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
df['date_val'] = pd.to_datetime(df['date'], format='mixed')
df.set_index('date_val').resample('ME')["cluster"].value_counts().unstack().plot(ax = ax);

### Quel est le contenu des clusters ?

Comment faire pour faire un résumé des clusters ? 

Une réponse : prendre la liste des mots les plus spécifiques

In [ ]:
df_w = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out()) 

In [ ]:
df_w[df["cluster"] == 4].max().sort_values()[-20:]

On peut améliorer :

- le choix des clusters
- la manière de résumer un cluster
- la représentation des textes
- le découpage des textes

## D'autres embeddings

Par exemple, les modèles fasttext https://fasttext.cc/

In [ ]:
from gensim.models.fasttext import load_facebook_vectors
import numpy as np

In [ ]:
# Télécharger ce modèle depuis https://fasttext.cc/docs/en/crawl-vectors.html
model = load_facebook_vectors('../../../../../../Documents/models/cc.fr.300.bin')

In [ ]:
def document_vector(words):
    word_vecs = [model[word] for word in words if word in model]
    return np.mean(word_vecs, axis=0) if word_vecs else np.zeros(model.vector_size)

# Matrice d'embeddings
X_embed = np.array([document_vector(doc) for doc in documents])
print(X_embed.shape)

Ou bien utiliser sbert https://sbert.net/ 

On voit apparaître de plus en plus de modèles

Ensuite la logique est la même...

## Empiler les traitements : BERTOPIC

Utiliser les différentes couches ensemble pour faire de la détection de thématiques : pipelines comme [BERTOPIC](https://maartengr.github.io/BERTopic/index.html)

![](../slides/img/bertopic.png){fig-align="center"}

[Faisons un test sur les données](https://maartengr.github.io/BERTopic/getting_started/quickstart/quickstart.html)


In [ ]:
import nltk
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
import bertopic

nltk.download("stopwords")
french_stopwords = list(set(stopwords.words("french")))
vectorizer_model = CountVectorizer(stop_words=french_stopwords)
topic_model = bertopic.BERTopic(            language="french", 
            vectorizer_model=vectorizer_model)
topics, probs = topic_model.fit_transform(df['texte'])

In [ ]:
topic_model.get_topic_info()